## 1.일반적 방식의 스태킹방법 사용

**- 데이터셋 , 필요모듈 불러오기 및 데이터 split**

In [22]:
import numpy as np


from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer_data = load_breast_cancer()

X_data = cancer_data.data
y_label = cancer_data.target

X_train , X_test , y_train , y_test = train_test_split(X_data , y_label , test_size=0.2 , random_state=0)


**-개별,메타 ML 모델을 위한 Classifier 생성**

In [23]:
#개별모델
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=100)

# 메타모델
lr_final = LogisticRegression()


**-개별모델 학습**

In [24]:
rf_clf.fit(X_train , y_train)
dt_clf.fit(X_train , y_train)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=100)

**-학습된 개별 모델들이 각자 반환하는 예측결과값을 변수에 따로 담아두고,  일차적으로 개별 모델의 정확도 측정**

In [33]:
# 학습된 개별 모델들이 각자 반환하는 예측 데이터 셋을 생성하고 개별 모델의 정확도 측정. 

rf_pred = rf_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)

print('랜덤 포레스트 정확도: {0:.4f}'.format(accuracy_score(y_test, rf_pred)))
print('결정 트리 정확도: {0:.4f}'.format(accuracy_score(y_test, dt_pred)))
print('에이다부스트 정확도: {0:.4f}'.format(accuracy_score(y_test, ada_pred)))

랜덤 포레스트 정확도: 0.9649
결정 트리 정확도: 0.9123
에이다부스트 정확도: 0.9561


**-예측결과 값을 stacking 한후 , 행과 열을 전치하여 -> 컬럼 레벨로 각 알고리즘의 예측 결과를 피처로 만듦**

In [54]:
#각각의 예측결과값을 하나의 예측배열로 stacking
pred = np.array([rf_pred, dt_pred, ada_pred])
print(pred.shape)

#전치
pred = np.transpose(pred)
print(pred.shape)

(3, 114)
(114, 3)


**-최종 메타 모델 학습/예측/평가**

In [38]:
lr_final.fit(pred, y_train) #과적합 발생위험
final = lr_final.predict(pred)

print('최종 메타 모델의 예측 정확도: {0:.4f}'.format(accuracy_score(y_test , final)))

최종 메타 모델의 예측 정확도: 0.9561


### > lr_final.fit(pred, y_test) # 여기서 메타모델(logistic_regression)의 학습시, 입력되는 데이터가 학습용 타겟값(y_train) 값이 아닌, 테스트 용 타겟값(y_test)가 들어갈수밖에 때문에(shape이 불일치) 모델예측시(predict) 과적합의 위험이 존재한다.

### > 따라서 최종 메타 모델의 학습때에도 train용 타겟값을 사용하기 위해 kFold로 데이터를 쪼개서 fit(x_train,y_train)이 성립될수 있도록 해준다.

## 2. CV 셋 기반의 Stacking

**-개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수 정의**

In [42]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

 
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds ):
    # 지정된 n_folds값으로 KFold 생성.
    kf = KFold(n_splits=n_folds, shuffle=False)
    #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화 
    train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ))
    test_pred = np.zeros((X_test_n.shape[0],n_folds)) 
    print(model.__class__.__name__ , ' model 시작 ') #classifier이름 출력
    
    
    #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출 
    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        
        print('\t 폴드 세트: ',folder_counter+1,' 시작 ')
        X_tr = X_train_n[train_index] 
        y_tr = y_train_n[train_index] 
        X_te = X_train_n[valid_index]  
        
        #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행.
        model.fit(X_tr , y_tr)       
        #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장. (메타모델에서 x_train용)
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장.  (메타모델에서 x_test용)
        test_pred[:, folder_counter] = model.predict(X_test_n)
            
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성 
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    
    
    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred , test_pred_mean

**-함수사용해서 메타모델이 사용할 데이터 추출**

In [43]:
rf_train, rf_test = get_stacking_base_datasets(rf_clf, X_train, y_train, X_test, 5)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, X_train, y_train, X_test,  5)    
ada_train, ada_test = get_stacking_base_datasets(ada_clf, X_train, y_train, X_test, 5)

RandomForestClassifier  model 시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
DecisionTreeClassifier  model 시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
AdaBoostClassifier  model 시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 


**-stacking후 y_train과 맞아 fit할수있는지 shape확인 + 메타모델 학습/예측/평가**

In [51]:
Stack_final_X_train = np.concatenate((rf_train, dt_train, ada_train), axis=1)
Stack_final_X_test = np.concatenate((rf_test, dt_test, ada_test), axis=1)
print(Stack_final_X_train.shape)
print(y_train.shape)
lr_final.fit(Stack_final_X_train, y_train)
stack_final = lr_final.predict(Stack_final_X_test)

print('최종 메타 모델의 예측 정확도: {0:.4f}'.format(accuracy_score(y_test, stack_final)))

(455, 3)
(455,)
최종 메타 모델의 예측 정확도: 0.9825
